In [1]:
from Ownkey import API_Key
import pandas as pd
import requests


def fetch_crypto_data(API_Key, tickers, multiplier=1, timespan="day", start_date="2024-01-09", end_date="2024-12-10"):
    # Initialize an empty list to collect data
    crypto_data = []
    
    # Loop through each ticker
    for ticker in tickers:
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{start_date}/{end_date}?apiKey={API_Key}"
        response = requests.get(url)
        data = response.json()
        
        if "results" in data:
            # Create a DataFrame from the results
            df = pd.DataFrame(data["results"])
            df['t'] = pd.to_datetime(df['t'], unit='ms')  # Convert timestamp to datetime
            df.rename(columns={'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v': 'Volume'}, inplace=True)
            
            # Get the latest row and make a copy
            latest_row = df.iloc[-1].copy() 
            # Append the ticker and format the row for final output
            crypto_data.append({
                "Ticker": ticker,
                "Date": latest_row["t"],
                "Open": latest_row["Open"],
                "High": latest_row["High"],
                "Low": latest_row["Low"],
                "Close": latest_row["Close"],
                "Volume": latest_row["Volume"]
            })
        else:
            print(f"Error fetching data for {ticker}: {data}")
    
    # Combine all data into a single DataFrame
    final_df = pd.DataFrame(crypto_data)
    
    return final_df

# Example usage
tickers = ["X:BTCUSD", "X:ETHUSD", "X:LTCUSD"]  # Add more tickers here

try:
    df = fetch_crypto_data(API_Key, tickers)
    print(df)
except ValueError as e:
    print(e)

/Users/edgarrios/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


     Ticker       Date      Open      High       Low     Close         Volume
0  X:BTCUSD 2024-12-10  97330.66  98368.00  94273.38  96660.76   23846.849456
1  X:ETHUSD 2024-12-10   3716.27   3785.00   3516.00   3630.34  266795.005604
2  X:LTCUSD 2024-12-10    110.48    114.97    101.14    109.92  935977.426638


In [2]:
df

,Ticker,Date,Open,High,Low,Close,Volume
0,X:BTCUSD,2024-12-10,97330.66,98368.00,94273.38,96660.76,23846.849456
1,X:ETHUSD,2024-12-10,3716.27,3785.00,3516.00,3630.34,266795.005604
2,X:LTCUSD,2024-12-10,110.48,114.97,101.14,109.92,935977.426638


In [11]:
new_df = df.drop(["Ticker", "Date"], axis=1)
new_df

,Open,High,Low,Close,Volume
0,97330.66,98368.00,94273.38,96660.76,23846.849456
1,3716.27,3785.00,3516.00,3630.34,266795.005604
2,110.48,114.97,101.14,109.92,935977.426638


In [12]:
new_df.corr()

,Open,High,Low,Close,Volume
Open,1.000000,1.000000,1.000000,1.000000,-0.728681
High,1.000000,1.000000,1.000000,1.000000,-0.728843
Low,1.000000,1.000000,1.000000,1.000000,-0.728172
Close,1.000000,1.000000,1.000000,1.000000,-0.728295
Volume,-0.728681,-0.728843,-0.728172,-0.728295,1.000000
